In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from tqdm import tqdm

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
latent_dim = 100
img_size = 28
channels = 1
batch_size = 64
num_epochs_gan = 20
num_epochs_classifier = 5
learning_rate = 0.0002

# Image shape
img_shape = (channels, img_size, img_size)

# GAN Components: Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, int(torch.prod(torch.tensor(img_shape)))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

# GAN Components: Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(int(torch.prod(torch.tensor(img_shape))), 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# MNIST Classifier
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        return self.model(x)

# Data loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

mnist = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
data_loader = DataLoader(mnist, batch_size=batch_size, shuffle=True)

# Initialize GAN models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
adversarial_loss = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)

# Training GAN
for epoch in tqdm(range(num_epochs_gan)):
    for i, (imgs, _) in enumerate(data_loader):
        # Prepare real and fake inputs
        real_imgs = imgs.to(device)
        batch_size = real_imgs.size(0)
        valid = torch.ones((batch_size, 1), device=device, dtype=torch.float32)
        fake = torch.zeros((batch_size, 1), device=device, dtype=torch.float32)

        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(batch_size, latent_dim, device=device)
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

    print(f"[Epoch {epoch+1}/{num_epochs_gan}] Loss D: {d_loss.item()}, Loss G: {g_loss.item()}")

# Save some generated images
z = torch.randn(64, latent_dim, device=device)
generated_imgs = generator(z)
save_image(generated_imgs.data, "generated_images.png", nrow=8, normalize=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 21.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 617kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

[Epoch 18/20] Loss D: 0.1348954439163208, Loss G: 4.629648685455322


 95%|█████████▌| 19/20 [11:54<00:37, 37.33s/it]

[Epoch 19/20] Loss D: 0.14929147064685822, Loss G: 3.368868350982666


100%|██████████| 20/20 [12:32<00:00, 37.62s/it]

[Epoch 20/20] Loss D: 0.20883461833000183, Loss G: 3.9148964881896973


In [3]:
# Train MNIST Classifier
classifier = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer_C = optim.Adam(classifier.parameters(), lr=learning_rate)

# Prepare dataset combining real and synthetic images
synthetic_imgs = generator(torch.randn(len(mnist), latent_dim, device=device)).detach()
synthetic_labels = torch.randint(0, 10, (len(mnist),), device=device)  # Random labels for GAN outputs
combined_data = torch.cat([mnist.data.unsqueeze(1).float() / 255.0, synthetic_imgs])
combined_labels = torch.cat([mnist.targets, synthetic_labels])

combined_dataset = torch.utils.data.TensorDataset(combined_data, combined_labels)
combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)

# Training the classifier
for epoch in tqdm(range(20)):
    for imgs, labels in combined_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer_C.zero_grad()
        outputs = classifier(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_C.step()

    print(f"[Epoch {epoch+1}/{num_epochs_classifier}] Loss: {loss.item()}")

# Evaluate classifier on real MNIST test set
test_mnist = datasets.MNIST(root="./data", train=False, transform=transform)
test_loader = DataLoader(test_mnist, batch_size=batch_size, shuffle=False)

correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = classifier(imgs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Classifier accuracy on real MNIST test set: {100 * correct / total:.2f}%")

  5%|▌         | 1/20 [00:09<03:03,  9.64s/it]

[Epoch 1/5] Loss: 1.3152340650558472


 10%|█         | 2/20 [00:19<02:58,  9.92s/it]

[Epoch 2/5] Loss: 1.0518131256103516


 15%|█▌        | 3/20 [00:30<02:52, 10.12s/it]

...

 95%|█████████▌| 19/20 [03:25<00:11, 11.25s/it]

[Epoch 19/5] Loss: 1.0142955780029297


100%|██████████| 20/20 [03:36<00:00, 10.83s/it]

[Epoch 20/5] Loss: 1.364072561264038


Classifier accuracy on real MNIST test set: 16.87%
